In [132]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [133]:
# this version is more transferable, as compared to the next cell (which relies on path on my computer)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/annasours/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [18]:
# # Setup splinter
# executable_path = {'executable_path': '/Users/annasours/chromedriver/chromedriver'}

# # executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# from splinter import Browser
# import pymongo

In [134]:
# URLs of pages to be scraped

# NASA Mars News
url_1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
# JPL Mars Space Images - Featured Image
url_2 = "https://www.jpl.nasa.gov/images?search=&category=Mars"
# Mars Facts
url_3 = "https://space-facts.com/mars/"
# Mars Hemispheres
url_4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Nasa Mars News

In [ ]:
# Optional delay for loading the page --> accounts for slow internet connection or pictures that load.
#browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

#browser.is_element_present_by_text("Feedback", wait_time=1)

#browser.is_element_present_by_id("email", wait_time=1)

# browser.is_element_present_by_name("email", wait_time=1)

In [135]:
# Try again using broswer
# browser = init_browser()
browser.visit(url_1)

# one way to get the webpage to load slower
time.sleep(2)

In [136]:
html=browser.html
news_soup=BeautifulSoup(html,'html.parser')

In [137]:
first_article=news_soup.find('ul', class_='item_list')

In [141]:
first_article.find('div', class_='article_teaser_body').text

'Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.'

In [142]:
news_title=first_article.find('div', class_='content_title').a.text
news_p=first_article.find('div', class_='article_teaser_body').text

print(news_title)
print('-----'*5)
print(news_p)

NASA's Perseverance Mars Rover Mission Honors Navajo Language
-------------------------
Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.


### JPL Mars Space Image - Featured Image

In [112]:
browser.visit(url_2)

# one way to get the webpage to load slower
time.sleep(2)
# browser.links.find_by_partial_text(title).click()


In [113]:
html=browser.html
jpl_soup=BeautifulSoup(html,'html.parser')

galleries=jpl_soup.find('div', class_='-active')
type(galleries)

bs4.element.Tag

In [61]:
button=galleries.find('button')
button.click

TypeError: 'NoneType' object is not callable

In [66]:
button=browser.find_by_name('Galleries')

for each in button:
    each.click()

In [104]:
name=jpl_soup.find('div', class_='-active').text

In [106]:
print(name)
print(name.strip())


                  Galleries
                 
Galleries


In [109]:
browser.links.find_by_partial_text(name).click()
# browser.find_link_by_text('Galleries').first.click()

# len(browser.find_by_name('Galleries'))
# browser.links.find_by_partial_text('Galleries').first
# browser.links.find_by_partial_text('Galleries').click()
# browser.links.find_by_css('div[class_="-active"]').click()


ElementDoesNotExist: no elements could be found with link by partial text "
                  Galleries
                 "

In [41]:
for each in galleries:
    each.click()

TypeError: 'NoneType' object is not callable

In [8]:
# Filtering by galleries doesn't work, doesn't return anything (or a list with 0 items in it)
# galleries=browser.links.find_by_partial_text('Galleries')
# len(galleries)
# try filtering by mars then select the first image
mars=browser.links.find_by_partial_text('Mars')

In [11]:
type(mars)

splinter.element_list.ElementList

In [5]:
button=browser.links.find_by_partial_text('Featured')
# browser.click_link_by_text('Galleries')

In [6]:
button.click()

ElementDoesNotExist: no elements could be found with link by partial text "Featured"

#### Method test: use beautiful soup object and sort by marg, then select image

In [12]:
html=browser.html
jpl_soup=BeautifulSoup(html,'html.parser')

In [13]:
type(jpl_soup)

bs4.BeautifulSoup

In [22]:
img_soup=jpl_soup.find_all('img')

len(img_soup)
type(img_soup)
print(img_soup)

[<img alt="JPL Logo" height="72" src="/_nuxt/img/logo-tribrand-color.8f95c3b.svg" width="324"/>, <img alt="JPL Logo" height="72" src="/_nuxt/img/logo-tribrand-color.8f95c3b.svg" width="324"/>, <img alt="" class="BaseImage object-contain" data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24460.2e16d0ba.fill-400x400-c50.jpg" loading="lazy" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24460.2e16d0ba.fill-400x400-c50.jpg"/>, <img alt="" class="BaseImage object-contain" data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24459.2e16d0ba.fill-400x400-c50.jpg" loading="lazy" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24459.2e16d0ba.fill-400x400-c50.jpg"/>, <img alt="" class="BaseImage object-contain" data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24493.2e16d0ba.fill-400x400-c50.jpg" loading="lazy" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24493.2e16d0ba.fill-400x400-c50.jpg"/>, <img alt="" class="BaseImage object-con

In [21]:
for each_img in img_soup:
    title=each_img
    print(title)

#### Method test: find BaseImages

##### This works when the dropdown 'Galleries' menu is opened by hand. Returns the featured image url

In [122]:
html=browser.html
pic_soup=BeautifulSoup(html,'html.parser')
featured_image_url=pic_soup.find('img', class_='BaseImage')['src']

In [123]:
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg


In [121]:
# galleries=jpl_soup.find_all('div', class_='-active')

# # navbar=
# # .find('img')
# for each_one in galleries:
#     each_one.find('div', class_='NavDesktopDropdownContent')

#### Method test: find navbar contents, then loop through and if text matches 'Galleries', use browser to click

In [81]:
html = browser.html
jpl_soup = BeautifulSoup (html, 'html.parser')

navbar=jpl_soup.find_all('div', class_='NavDesktopDropdown')

# search for galleries button in navbar, then click
for each_item in navbar:
    name=each_item.text
    print(name)
    if name == 'Galleries':
        print('Button found!')
        


                  About JPL
                 

                  Missions
                 

                  News
                 

                  Galleries
                 

                  Engage
                 

                  More
                 


In [79]:
for each_item in navbar:
    button=each_item.find('button')
#     print(each_item)
    span=each_item.find('span')
    print(span)
    
#     try:
#         button.click()
#     except:
#         print('Error clicking button')

    try:
        span.click()
    except:
        print('Error clicking span')

<span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  About JPL
                </span>
Error clicking span
<span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  Missions
                </span>
Error clicking span
<span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  News
                </span>
Error clicking span
<span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  Galleries
                </span>
Error clicking span
<span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  Engage
                </span>
Error clicking span
<span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  More
                </span>
Error clicking span


In [102]:
# html = browser.html
# jpl_soup = BeautifulSoup (html, 'html.parser')

# # navbar=jpl_soup.find_all('div', class_='-active')
# navbar=jpl_soup.find_all('div', class_='NavDesktopDropdown')


# navbar
# # for each in navbar:   
# #     browser.links.find_by_partial_text('Galleries').click()

[<div class="NavDesktopDropdown"><button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                   About JPL
                 </span></button> <!-- --></div>,
 <div class="NavDesktopDropdown"><button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                   Missions
                 </span></button> <!-- --></div>,
 <div class="NavDesktopDropdown"><button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                   News
                 </span></button> <!-- --></div>,
 <div class="NavDesktopDropdown -active"><b

In [93]:
# # Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# # and assign the url string to a variable called featured_image_url.

# # galleries_button=jpl_soup.find_all('div', class_='NavDesktopDropdown')[3]

# # type(jpl_soup.find_all('div', class_='NavDesktopDropdown'))

# navbar=jpl_soup.find_all('div', class_='-active')

# # navbar=
# # .find('img')
# # navbar.click()

In [94]:
# navbar

[<div class="NavDesktopDropdown -active"><button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                   Galleries
                 </span></button> <!-- --></div>]

In [72]:
# # Use splinter to navigate the site and find the image url for the current Featured Mars Image \n",
# # and assign the url string to a variable called featured_image_url.\n",   
# navbar=jpl_soup.find_all('div', class_='NavDesktopDropdownContent')


In [73]:
# len(navbar)

0

In [108]:
# jpl_soup.find('img', class_='BaseImage')

<img alt="" class="BaseImage object-contain" data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24460.2e16d0ba.fill-400x400-c50.jpg" loading="lazy" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24460.2e16d0ba.fill-400x400-c50.jpg"/>

In [108]:
# navbar[3].find('button')

<button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  Galleries
                </span></button>

In [101]:
# for each_item in navbar:
#     print(each_item.find('button'))

<button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  About JPL
                </span></button>
<button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  Missions
                </span></button>
<button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in border-b-2">
                  News
                </span></button>
<button aria-expanded="false" aria-haspopup="true" class="group relative z-20 px-4 font-medium border-t-2 border-transparent"><span class="inline-block py-2 transition-colors duration-100 ease-in 

In [55]:
# galleries=browser.find_by_text('Galleries')
# print(galleries)

In [61]:
# browser.links.find_by_text('Galleries').click()


ElementDoesNotExist: no elements could be found with link by text "Galleries"

In [62]:
# galleries_button.a

In [56]:
# featured_image_url=navbar['data-src']
# featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg'

In [58]:
# is this necessary? 
# browser.links.find_by_text(featured_image_url)

### Mars Facts

In [128]:
import pandas as pd

In [129]:
tables = pd.read_html(url_3)
mars_facts_df=tables[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [130]:
html_table = mars_facts_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [66]:
# if more needed:
# strip unwanted newlines to clean it up
# html_table.replace('\n', '') 

The file /Users/annasours/Documents/Bootcamp/Work Here/12-Web-Scraping-and-Document-Databases/12-Web-Scraping-and-Document-Databases/web-scraping-challenge/Mission_to_Mars/html_table.html does not exist.


### Mars Hemispheres

In [124]:
browser.visit(url_4)
time.sleep(2)

In [125]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [126]:
photos = soup.find_all('div', class_='description')

In [127]:
len(photos)

4

In [128]:
print(photos)

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>, <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>, <div class="description"><a cl

In [129]:

base_url='https://astrogeology.usgs.gov'
img_dict_list=[]

# browser.visit(url_4)

for each_photo in photos:
#     find img_url
    h3 = each_photo.find('h3')
    link = each_photo.find('a')
    href = link['href']
    title=h3.text

    print(f'Getting {title} link')

#     this will click on each photo link.
    browser.links.find_by_partial_text(title).click()
    pic_html = browser.html
    pic_soup = BeautifulSoup(pic_html, 'html.parser')
    source_photo=pic_soup.find('img', class_="wide-image")['src']
    img_url=(base_url+source_photo)

#     create dictionary with title and img url and append to list of dictionaries
    img_dict_list.append({'title': title, 'img_url': img_url})
    
#     go back to original page to follow next link
    browser.visit(url_4)
    


Getting Cerberus Hemisphere Enhanced link
Getting Schiaparelli Hemisphere Enhanced link
Getting Syrtis Major Hemisphere Enhanced link
Getting Valles Marineris Hemisphere Enhanced link


In [130]:
img_dict_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]